In [13]:
import networkx as nx
import pandas as pd
import numpy as np
from scipy import stats
import statistics
import itertools

In [14]:
simplify_dir = '/Volumes/Data2/RST/notebook/GTFS/TTC_2016-10-03/GIS Simplify/'
gtfs_dir = '/Volumes/Data2/RST/notebook/GTFS/TTC_2016-10-03/'

In [15]:
ward_int = pd.read_csv('/Volumes/Data2/RST/notebook/GIS/int_wards.csv')
ward_int['INT_ID_STR'] = ward_int['INT_ID'].astype(str)

In [16]:
period = 'EM'
period_hr = 4

In [17]:
G = nx.read_gexf('/Volumes/Data2/RST/notebook/networks/' + period +'-RM-16-singleDi.gexf')
graph_data = pd.read_csv('/Volumes/Data2/RST/notebook/networks/' + period +'-RM-16-singleDi.csv')
stop_times_od = pd.read_csv(gtfs_dir + 'stop_times_full.csv')

In [18]:
route_dir = stop_times_od[stop_times_od['hr_o'] == period_hr]
route_dir = route_dir[['INT_ID_o', 'INT_ID_d', 'route_short_name']].drop_duplicates()
route_dir = route_dir.groupby(['INT_ID_o', 'INT_ID_d'])['route_short_name'].apply(list).reset_index()
route_dir = route_dir.merge(graph_data)

In [19]:
def func(row):

    single_cycle = row.copy()
    single_cycle.append(single_cycle[0])
    single_cycle.append(None)


    cycle_pairs = list(zip(single_cycle [::1], single_cycle [1::1]))[:-1]
    cycle_pairs = [list(map(int, sublist)) for sublist in cycle_pairs]
    cycle_df = route_dir.merge(pd.DataFrame.from_records(cycle_pairs, columns = ['INT_ID_o', 'INT_ID_d']))

    first_route = cycle_df.iloc[0,2]

    #check for cycles where it can be traversed by 1 route, ie eliminate out and back trips
    for route in first_route:
        if len(cycle_df[cycle_df['route_short_name'].apply(lambda x: route in x)]) == len(cycle_df):
            return ward,0,0,1
            break
        else:
            return ward, 1, cycle_df['scaled_freq'].min(), 0

In [20]:
result_list = []
ward_n = []

for ward in range(1,26,1):

    G_ward = G.copy()


    G_ward.remove_nodes_from(list(ward_int[ward_int['AREA_SHORT_CODE'] != ward]['INT_ID_STR']))

    N = G_ward.size()

    #find cycles
    cycle_list = list(nx.simple_cycles(G_ward))
    print('Ward ' + str(ward) + ', Size ' + str(N) + ' Found '+ str(len(cycle_list)) + ' cycles')

    temp_list = list(map(func, cycle_list))

    result_list.append(temp_list)
    ward_n.append([ward, N])


Ward 1, Size 207 Found 104 cycles
Ward 2, Size 110 Found 43 cycles
Ward 3, Size 175 Found 81 cycles
Ward 4, Size 103 Found 52 cycles
Ward 5, Size 197 Found 131 cycles
Ward 6, Size 146 Found 74 cycles
Ward 7, Size 102 Found 40 cycles
Ward 8, Size 97 Found 41 cycles
Ward 9, Size 106 Found 47 cycles
Ward 10, Size 65 Found 32 cycles
Ward 11, Size 47 Found 23 cycles
Ward 12, Size 40 Found 18 cycles
Ward 13, Size 83 Found 44 cycles
Ward 14, Size 141 Found 77 cycles
Ward 15, Size 66 Found 31 cycles
Ward 16, Size 72 Found 35 cycles
Ward 17, Size 100 Found 46 cycles
Ward 18, Size 97 Found 44 cycles
Ward 19, Size 45 Found 20 cycles
Ward 20, Size 118 Found 42 cycles
Ward 21, Size 110 Found 44 cycles
Ward 22, Size 110 Found 56 cycles
Ward 23, Size 108 Found 52 cycles
Ward 24, Size 67 Found 32 cycles
Ward 25, Size 106 Found 39 cycles


In [21]:
combined = list(itertools.chain.from_iterable(result_list))
result_df = pd.DataFrame.from_records(
    combined, columns = ['ward', 'cycles', 'scaled_cycles', 'removed_cycles']).groupby(
    'ward').sum().reset_index()
result_df = result_df.merge(pd.DataFrame.from_records(ward_n, columns = ['ward', 'N']))
result_df

,ward,cycles,scaled_cycles,removed_cycles,N
0,1,19,0.2500,85,207
1,2,1,0.0125,42,110
2,3,8,0.1500,73,175
3,4,3,0.0750,49,103
4,5,49,0.9250,82,197
5,6,12,0.1750,62,146
6,7,3,0.0500,37,102
7,8,2,0.0500,39,97
8,9,5,0.0625,42,106
9,10,3,0.0750,29,65


In [22]:
result_df['cycle_availability'] = result_df['scaled_cycles']/(2 * result_df['N'] + 5)

In [23]:
result_df = result_df.merge(ward_int[['AREA_NAME', 'AREA_SHORT_CODE']].drop_duplicates(), left_on = ['ward'], 
                 right_on = ['AREA_SHORT_CODE']).rename(columns = {'AREA_NAME':'ward_name'}).drop(
    columns = ['AREA_SHORT_CODE'])
result_df.sort_values(by = 'cycle_availability', ascending = False)

,ward,cycles,scaled_cycles,removed_cycles,N,cycle_availability,ward_name
4,5,49,0.9250,82,197,0.002318,York South-Weston
13,14,13,0.3250,64,141,0.001132,Toronto-Danforth
12,13,6,0.1500,38,83,0.000877,Toronto Centre
22,23,7,0.1750,45,108,0.000792,Scarborough North
18,19,1,0.0750,19,45,0.000789,Beaches-East York
21,22,5,0.1500,51,110,0.000667,Scarborough-Agincourt
0,1,19,0.2500,85,207,0.000597,Etobicoke North
5,6,12,0.1750,62,146,0.000589,York Centre
9,10,3,0.0750,29,65,0.000556,Spadina-Fort York
2,3,8,0.1500,73,175,0.000423,Etobicoke-Lakeshore


In [25]:
result_df.to_csv('cycles-' + period + '.csv', index = False)